In [1]:
import pandas as pd
import numpy as np

# Random seed for reproducibility
np.random.seed(42)

# Number of samples
num_samples = 1000

# Feature generation
fossil_sizes = np.random.uniform(1, 50, num_samples)  # Fossil sizes between 1 and 50 cm
types = np.random.choice(['Mammal', 'Reptile', 'Bird', 'Fish', 'Insect'], num_samples)
layer_depths = np.random.uniform(0.5, 20, num_samples)  # Geological layer depths between 0.5 and 20 meters
locations = np.random.choice(['North America', 'South America', 'Europe', 'Asia', 'Africa', 'Australia'], num_samples)

# Adjust ages based on type and location
type_age_adjustments = {
    'Mammal': -10,
    'Reptile': 0,
    'Bird': -5,
    'Fish': -2,
    'Insect': -3
}

location_age_adjustments = {
    'North America': -5,
    'South America': -10,
    'Europe': -15,
    'Asia': -20,
    'Africa': -25,
    'Australia': -5
}

# Generate fossil ages with adjustments in the initial calculation
ages = (layer_depths * 5 + fossil_sizes * 0.2 + 
        np.array([type_age_adjustments[t] for t in types]) +
        np.array([location_age_adjustments[l] for l in locations]) +
        np.random.normal(0, 2, num_samples)).astype(int)

# Create the DataFrame
df = pd.DataFrame({
    'Fossil_Size': fossil_sizes,
    'Type': types,
    'Layer_Depth': layer_depths,
    'Location': locations,
    'Age': ages
})

# Save the dataset to a CSV file
df.to_csv('fossil_age_dataset.csv', index=False)
print("Dataset created and saved as 'fossil_age_dataset.csv'")


Dataset created and saved as 'fossil_age_dataset.csv'


In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Load dataset
df = pd.read_csv('fossil_age_dataset.csv')

# Feature selection
features = ['Fossil_Size', 'Type', 'Layer_Depth', 'Location']
target = 'Age'

# Data preprocessing
X = pd.get_dummies(df[features])  # One-hot encode categorical features
y = df[target]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.09, random_state=42)

# Train the Linear Regression model
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Evaluate the Linear Regression model
linear_predictions = linear_model.predict(X_test)
linear_mse = mean_squared_error(y_test, linear_predictions)
print(f'Linear Regression Mean Squared Error: {linear_mse}')


Linear Regression Mean Squared Error: 3.7385474975931783


In [ ]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk, ImageOps  # Ensure you have Pillow installed
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Load dataset and train model
df = pd.read_csv('fossil_age_dataset.csv')
features = ['Fossil_Size', 'Type', 'Layer_Depth', 'Location']
target = 'Age'
X = pd.get_dummies(df[features])  # One-hot encode categorical features
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(X_train, y_train)

class FossilAgeApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Fossil Age Predictor")
        self.root.geometry("560x430")  # Set fixed window size
        self.root.resizable(False, False)  # Disable window resizing
        self.root.configure(bg='saddlebrown')
        
        # Make the grid expand with the window
        for i in range(7):
            self.root.grid_rowconfigure(i, weight=1)
        for i in range(6):
            self.root.grid_columnconfigure(i, weight=1)

        # Define custom style for sliders
        style = ttk.Style()
        style.configure("TScale", background='saddlebrown', troughcolor='saddlebrown')
        
        # Main frame
        main_frame = tk.Frame(root, bg='saddlebrown')
        main_frame.grid(row=0, column=0, padx=10, pady=10, sticky='nsew')
        
        # Title
        self.title_label = tk.Label(main_frame, text="Fossil Age Predictor", font="Arial 24 bold", bg='saddlebrown', fg='white')
        self.title_label.grid(row=0, column=0, columnspan=6, pady=10)
        
        # Fossil Size
        tk.Label(main_frame, text="Fossil Size (cm)", font="Arial 14", bg='saddlebrown', fg='white').grid(row=1, column=0, pady=5, sticky='e')
        self.size_value = tk.Label(main_frame, text="25", font="Arial 12", bg='saddlebrown', fg='white')
        self.size_value.grid(row=1, column=5, pady=5, sticky='w')
        self.size_scale = ttk.Scale(main_frame, from_=1, to=50, orient=tk.HORIZONTAL, style="TScale", command=lambda v: self.size_value.config(text=f"{float(v):.1f}"))
        self.size_scale.set(25)  # Initial value
        self.size_scale.grid(row=1, column=1, columnspan=4, pady=5, sticky='we')
        
        # Type
        tk.Label(main_frame, text="Type", font="Arial 14", bg='saddlebrown', fg='white').grid(row=2, column=0, pady=5, sticky='e')
        self.type_var = tk.StringVar()
        self.type_frame = tk.Frame(main_frame, bg='saddlebrown')
        self.type_frame.grid(row=2, column=1, columnspan=5, pady=5, sticky='w')
        
        # Load images
        self.images = {}
        self.selected_images = {}
        types = ['Mammal', 'Reptile', 'Bird', 'Fish', 'Insect']
        for type_ in types:
            image = Image.open(f"{type_.lower()}.png").resize((50, 50))
            monochrome_image = ImageOps.colorize(ImageOps.grayscale(image), black="saddlebrown", white="black").convert("RGBA")
            self.images[type_] = ImageTk.PhotoImage(monochrome_image)
            self.selected_images[type_] = ImageTk.PhotoImage(image.resize((60, 60)))

        self.image_labels = {}
        for type_ in types:
            label = tk.Label(self.type_frame, image=self.images[type_], text=type_, compound='top', bg='saddlebrown', fg='white', cursor='hand2')
            label.pack(side=tk.LEFT, padx=5)
            label.bind("<Button-1>", lambda e, type_=type_: self.select_type(type_))
            self.image_labels[type_] = label
        
        self.type_var.set(types[0])
        self.select_type(types[0])
        
        # Layer Depth
        tk.Label(main_frame, text="Layer Depth (m)", font="Arial 14", bg='saddlebrown', fg='white').grid(row=3, column=0, pady=5, sticky='e')
        self.depth_value = tk.Label(main_frame, text="10.0", font="Arial 12", bg='saddlebrown', fg='white')
        self.depth_value.grid(row=3, column=5, pady=5, sticky='w')
        self.depth_scale = ttk.Scale(main_frame, from_=0.5, to=20, orient=tk.HORIZONTAL, style="TScale", command=lambda v: self.depth_value.config(text=f"{float(v):.1f}"))
        self.depth_scale.set(10)  # Initial value
        self.depth_scale.grid(row=3, column=1, columnspan=4, pady=5, sticky='we')
        
        # Location
        tk.Label(main_frame, text="Location", font="Arial 14", bg='saddlebrown', fg='white').grid(row=4, column=0, pady=5, sticky='e')
        self.location_var = tk.StringVar()
        self.location_dropdown = ttk.Combobox(main_frame, textvariable=self.location_var, values=['North America', 'South America', 'Europe', 'Asia', 'Africa', 'Australia'], state='readonly')
        self.location_dropdown.grid(row=4, column=1, columnspan=5, pady=5, sticky='we')
        self.location_dropdown.current(0)
        
        # Prediction button
        self.predict_button = tk.Button(main_frame, text="Predict Age", font="Arial 14 bold", command=self.predict, bg='burlywood', fg='black')
        self.predict_button.grid(row=5, column=0, columnspan=6, pady=20, sticky='we')
        
        # Result display
        self.result_label = tk.Label(main_frame, text="Predicted Age: ", font="Arial 14 bold", bg='saddlebrown', fg='white')
        self.result_label.grid(row=6, column=0, columnspan=6, pady=10, sticky='we')
    
    def select_type(self, selected_type):
        for type_, label in self.image_labels.items():
            if type_ == selected_type:
                label.config(image=self.selected_images[type_])
            else:
                label.config(image=self.images[type_])
        self.type_var.set(selected_type)
    
    def predict(self):
        # Collect input data
        input_data = {
            'Fossil_Size': [self.size_scale.get()],
            'Type': [self.type_var.get()],
            'Layer_Depth': [self.depth_scale.get()],
            'Location': [self.location_var.get()]
        }
        
        # Create DataFrame for input
        input_df = pd.DataFrame(input_data)
        input_df = pd.get_dummies(input_df).reindex(columns=X.columns, fill_value=0)
        
        # Make prediction
        prediction = model.predict(input_df)
        self.result_label.config(text=f"Predicted Age: {prediction[0]} million years")

if __name__ == "__main__":
    root = tk.Tk()
    app = FossilAgeApp(root)
    root.mainloop()
